# Read Data

In [1]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('SUI_USDT_futures_15m_20241231.csv')

# Drop unused columns
df.drop(columns=['coin','unused', 'closetime'], inplace=True)

# Drop unused columns
df.drop(columns=['volume','quotevolume', 'trades', 'taker_buy_volume', 'taker_buy_quote'], inplace=True)

# Display the first few rows of the DataFrame
df

,opentime,openprice,highprice,lowprice,closeprice
0,1683129600000,1.4030,1.4030,1.2652,1.3338
1,1683130500000,1.3339,1.3863,1.3327,1.3716
2,1683131400000,1.3717,1.3957,1.3547,1.3857
3,1683132300000,1.3866,1.3924,1.3652,1.3733
4,1683133200000,1.3731,1.3756,1.3405,1.3574
...,...,...,...,...,...
58323,1735620300000,4.0787,4.0850,4.0616,4.0828
58324,1735621200000,4.0829,4.0953,4.0738,4.0810
58325,1735622100000,4.0811,4.0990,4.0637,4.0982
58326,1735623000000,4.0982,4.1310,4.0900,4.1198


# Explore

In [25]:
df_setup = df.copy()

# Calculate candle body size
df_setup['body_size'] = abs(df_setup['closeprice'] - df_setup['openprice'])

# Calculate tail length
df_setup['high_tail'] = abs(df_setup['highprice'] - df_setup[['openprice', 'closeprice']].max(axis=1))
df_setup['low_tail'] = abs(df_setup['lowprice'] - df_setup[['openprice', 'closeprice']].min(axis=1))

# Calculate tail to body ratio
df_setup['high_tail_ratio'] = df_setup['high_tail'] / df_setup['body_size']
df_setup['low_tail_ratio'] = df_setup['low_tail'] / df_setup['body_size']

# Calculate body + tail
df_setup['body_high_tail_ratio'] = (df_setup['high_tail'] + df_setup['body_size']) / df_setup['openprice'] * 100
df_setup['body_low_tail_ratio'] = (df_setup['low_tail'] + df_setup['body_size']) / df_setup['openprice'] * 100

# Get next open price
df_setup['next_openprice'] = df_setup['openprice'].shift(-1)

df_setup

,opentime,openprice,highprice,lowprice,closeprice,body_size,high_tail,low_tail,high_tail_ratio,low_tail_ratio,body_high_tail_ratio,body_low_tail_ratio,next_openprice
0,1683129600000,1.4030,1.4030,1.2652,1.3338,0.0692,0.0000,0.0686,0.000000,0.991329,4.932288,9.821810,1.3339
1,1683130500000,1.3339,1.3863,1.3327,1.3716,0.0377,0.0147,0.0012,0.389920,0.031830,3.928330,2.916261,1.3717
2,1683131400000,1.3717,1.3957,1.3547,1.3857,0.0140,0.0100,0.0170,0.714286,1.214286,1.749654,2.259969,1.3866
3,1683132300000,1.3866,1.3924,1.3652,1.3733,0.0133,0.0058,0.0081,0.436090,0.609023,1.377470,1.543343,1.3731
4,1683133200000,1.3731,1.3756,1.3405,1.3574,0.0157,0.0025,0.0169,0.159236,1.076433,1.325468,2.374190,1.3570
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58323,1735620300000,4.0787,4.0850,4.0616,4.0828,0.0041,0.0022,0.0171,0.536585,4.170732,0.154461,0.519773,4.0829
58324,1735621200000,4.0829,4.0953,4.0738,4.0810,0.0019,0.0124,0.0072,6.526316,3.789474,0.350241,0.222881,4.0811
58325,1735622100000,4.0811,4.0990,4.0637,4.0982,0.0171,0.0008,0.0174,0.046784,1.017544,0.438607,0.845360,4.0982
58326,1735623000000,4.0982,4.1310,4.0900,4.1198,0.0216,0.0112,0.0082,0.518519,0.379630,0.800351,0.727149,4.1199


In [37]:
import pandas as pd
import numpy as np

# Initialize parameters for iteration
tr_ratios = np.arange(1, 6, 1).tolist()
bt_ratios = np.arange(0.5, 0.9, 0.1).tolist()
maker_fee = 0.05  # Maker fee for the trades
results = []  # To store the results

# Outer loop for tr_ratio
for tr_ratio in tr_ratios:
    # Inner loop for bt_ratio
    for bt_ratio in bt_ratios:

        # Add a new column 'next_trend' and set default value to 0
        df_setup['next_trend'] = 0
        df_setup.loc[(df_setup['high_tail_ratio'] >= tr_ratio) & 
                     (df_setup['low_tail_ratio'] < tr_ratio) & 
                     (df_setup['body_high_tail_ratio'] >= bt_ratio), 'next_trend'] = -1
        df_setup.loc[(df_setup['low_tail_ratio'] >= tr_ratio) & 
                     (df_setup['high_tail_ratio'] < tr_ratio) & 
                     (df_setup['body_low_tail_ratio'] >= bt_ratio), 'next_trend'] = 1

        df_setup.dropna(inplace=True)

        # Initialize variables for trend alternation
        indices = []  # To store the captured indices and alternating trends
        previous_trend = None
        previous_index = None

        # Iterate through the DataFrame
        for idx, row in df_setup.iterrows():
            current_trend = row['next_trend']  # Current value of next_trend

            if previous_trend is None:
                if current_trend in [1, -1]:
                    previous_trend = current_trend
                    previous_index = idx
            else:
                if previous_trend == 1 and current_trend == -1:
                    indices.append((previous_trend, previous_index, current_trend, idx))
                    previous_trend = -1
                    previous_index = idx
                elif previous_trend == -1 and current_trend == 1:
                    indices.append((previous_trend, previous_index, current_trend, idx))
                    previous_trend = 1
                    previous_index = idx

        # Convert the captured indices into a DataFrame
        result_df = pd.DataFrame(indices, columns=['Start_Trend', 'Start_Index', 'End_Trend', 'End_Index'])

        # Filter df_setup based on Start_Index from result_df
        filtered_df_setup = df_setup.loc[result_df['Start_Index']]

        # Reset the index
        filtered_df_setup.reset_index(drop=True, inplace=True)

        # Backtest DataFrame
        df_backtest = filtered_df_setup.copy()
        df_backtest['pnl'] = (df_backtest['next_openprice'] - df_backtest['next_openprice'].shift(1)) / df_backtest['next_openprice'].shift(1) * 100
        df_backtest['pnl'] = np.where(df_backtest['next_trend'] == 1, df_backtest['pnl'] * -1, df_backtest['pnl'])

        # Calculate the sum of PnL
        total_pnl = df_backtest['pnl'].sum()

        # Count the rows in df_backtest
        row_count = len(df_backtest)

        # Calculate the total rows in df_setup for this combination
        total_rows_in_setup = len(df_setup)

        # Calculate win rate
        wins = df_backtest.apply(lambda row: 1 if ((row['pnl'] > 0 and row['next_trend'] == -1) or (row['pnl'] < 0 and row['next_trend'] == 1)) else 0, axis=1)
        win_rate = wins.sum() / row_count * 100 if row_count > 0 else 0

        # Calculate net PnL sum (subtracting fees)
        net_pnl_sum = total_pnl - row_count * maker_fee * 2

        # Calculate the ratio of df_backtest rows to df_setup rows
        backtest_to_setup_ratio = row_count / total_rows_in_setup * 100 if total_rows_in_setup > 0 else 0

         # Calculate score
        score = net_pnl_sum * backtest_to_setup_ratio

        # Append results to the list
        results.append({'tr_ratio': tr_ratio, 'bt_ratio': bt_ratio, 'pnl_sum': total_pnl, 'row_count': row_count, 
                        'win_rate': win_rate, 'net_pnl_sum': net_pnl_sum, 'backtest_to_setup_ratio': backtest_to_setup_ratio, 'score': score})


# Convert results to a DataFrame
results_df = pd.DataFrame(results)

# Display the results
results_df


,tr_ratio,bt_ratio,pnl_sum,row_count,win_rate,net_pnl_sum,backtest_to_setup_ratio,score
0,1,0.5,106.092147,4354,49.563620,-329.307853,7.465962,-2458.600076
1,1,0.6,26.402376,3350,49.671642,-308.597624,5.744367,-1772.698036
2,1,0.7,39.952921,2612,50.076570,-221.247079,4.478892,-990.941683
3,1,0.8,84.893931,1998,49.449449,-114.906069,3.426043,-393.673181
4,2,0.5,155.938558,2413,49.647741,-85.361442,4.137659,-353.196544
5,2,0.6,78.025469,1756,51.309795,-97.574531,3.011077,-293.804446
6,2,0.7,275.552444,1296,51.311728,145.952444,2.222298,324.349887
7,2,0.8,342.532370,948,50.527426,247.732370,1.625570,402.706345
8,3,0.5,161.306839,1611,50.279330,0.206839,2.762440,0.571381
9,3,0.6,219.680414,1148,51.132404,104.880414,1.968517,206.458924
